<a href="https://colab.research.google.com/github/Sekinat95/ML-study-linreg/blob/main/Copy_1_of_BC_w_openai_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 112kB 30.4MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, MaxPooling2D, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential
import tensorflow as tf
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
import gym
import tensorflowjs as tfjs

In [ ]:
!pip install tensorflow-gpu==2.0.0-rc0

In [ ]:
from google.colab import drive
import pickle
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# FETCHES THE DATA 

In [ ]:
#X = pickle.load(open('C:\\Users\sekin\\Downloads\\IMITATION LEARNING\\ML Engineer testing task-20210121T205321Z-001\\ML Engineer testing task\\ml-engineer-testing-task-data\\ml-engineer-testing-task\\data\\my_test_folder\\X_gray_full.pickle', "rb"))
X = pickle.load(open('/content/drive/MyDrive/data/X_gray_full.pickle', "rb"))[0:100000]
#y = np.load('C:\\Users\sekin\Downloads\\IMITATION LEARNING\\ML Engineer testing task-20210121T205321Z-001\\ML Engineer testing task\\ml-engineer-testing-task-data\\ml-engineer-testing-task\\data\\my_test_folder\\actions.npy').flatten()
y = np.load('/content/drive/MyDrive/data/actions.npy').flatten()[0:100000]

In [ ]:
X = X/255
#y = y.reshape(y.shape[0],-1).T
y = y.reshape(len(y), 1)
print(X.shape)
print(y.shape)

(100000, 84, 84, 1)
(100000, 1)


In [ ]:
#NAME = "BC-openaiGYM-cnn-64x2-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='C:\\Users\\sekin\\Downloads\\IMITATION LEARNING\\ML Engineer testing task-20210121T205321Z-001\\ML Engineer testing task\\ml-engineer-testing-task-data\\ml-engineer-testing-task\\data\\my_test_folder\\logs{}'.format(NAME))

In [ ]:
#----------------------------------------#
# create model#
#----------------------------------------#
# #def create_model():
model = Sequential()
# conv 1
model.add(Conv2D(64, (3, 3), input_shape=X[0].shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# conv 2
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# conv 3
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# fc 1
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.25))
model.add(Activation('relu'))
# fc 2
model.add(Dense(64))
model.add(Activation('relu'))
#out==logits
model.add(Dense(4))
model.add(Activation('softmax'))


model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

history = model.fit(x=X, y=y, batch_size=128, epochs=25,
          validation_split=0.2)

model.save('/content/drive/MyDrive/data/BC-openAI-CNN-11-ol.h5')

#saves the model as a converted tensorflowjs model
tfjs.converters.save_keras_model(model, '/content/drive/MyDrive/data/model-js-version1')

Train on 80000 samples, validate on 20000 samples
Epoch 1/25
80000/80000 [==============================] - 34s 421us/sample - loss: 1.2063 - accuracy: 0.4344 - val_loss: 1.1700 - val_accuracy: 0.4615
Epoch 2/25
80000/80000 [==============================] - 26s 324us/sample - loss: 1.1372 - accuracy: 0.4910 - val_loss: 1.1365 - val_accuracy: 0.4960
Epoch 3/25
80000/80000 [==============================] - 26s 326us/sample - loss: 1.0986 - accuracy: 0.5199 - val_loss: 1.1171 - val_accuracy: 0.5094
Epoch 4/25
80000/80000 [==============================] - 27s 332us/sample - loss: 1.0657 - accuracy: 0.5394 - val_loss: 1.0988 - val_accuracy: 0.5135
Epoch 5/25
80000/80000 [==============================] - 26s 330us/sample - loss: 1.0417 - accuracy: 0.5522 - val_loss: 1.0837 - val_accuracy: 0.5270
Epoch 6/25
80000/80000 [==============================] - 26s 329us/sample - loss: 1.0216 - accuracy: 0.5618 - val_loss: 1.0808 - val_accuracy: 0.5274
Epoch 7/25
80000/80000 [====================

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
def plot_learning_curve(history, epochs):
    """
    plots the loss and accuracy graphs for both the training 
    and validation sets
    """
    # train and eval accuracy
    epoch_range = range(1, epochs+1)
    plt.plot(epoch_range, history.history['accuracy'])
    plt.plot(epoch_range, history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    # train and eval loss
    epoch_range = range(1, epochs+1)
    plt.plot(epoch_range, history.history['loss'])
    plt.plot(epoch_range, history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
plot_learning_curve(history, 25)

**SHOW THE RESULT BY USING THE MODEL IN THE GYM ENVIRONMENT**


In [ ]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym[atari]
!pip install pyvirtualdisplay
!pip install piglet

In [ ]:
#model = tf.keras.models.load_model('/content/drive/MyDrive/data/BC-openAI-CNN-8-ol.h5')

**utility functions to convert the game environment to the format understood by the model**

In [ ]:
def grayConversion(image):
    grayValue = 0.07 * image[:,:,2] + 0.72 * image[:,:,1] + 0.21 * image[:,:,0]
    gray_img = grayValue.astype(np.float64)
    return gray_img
def prepare(obs):
    IMG_SIZE = 84
    # img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    obs = obs/255
    gray_obs =  grayConversion(obs)
    print("shape of gray_obs{}".format(gray_obs.shape))
    new_obs = cv2.resize(gray_obs, (IMG_SIZE, IMG_SIZE))
    print("shape of new_obs{}".format(new_obs.shape))
    return new_obs.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


**SETTING UP THE GYM ENVIRONMENT ON GOOGLE COLAB**

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
"""
where the model is used
"""
import gym
import matplotlib.pyplot as plt

env = gym.make("Breakout-v0")
env = wrap_env(env)

#
# explore MountainCar environment
#
plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

#
# take random actions and show the video result
# In MountainCar, observation is just two numbers: 
# car position and velocity
#
observation = env.reset()
print("initial observation code:", observation)

    
while True:
  env.render()
  
  # your agent goes here
  action = env.action_space.sample()   # take a random action
  prediction = model.predict_classes(prepare(observation))
  #print(prediction)
  observation, reward, done, info = env.step(prediction) 
  #print(reward)
   
  if done: 
    break;
            
env.close()
show_video()